# Cheryl's Birthday

> Everyone knows that Cheryl was born in February, March, or April.
>
> Cheryl separately tells Alice the month and Bob the day. Then they have this dialogue:
>
> 1. Alice: "I don't know when Cheryl's birthday is..."
> 2. Alice: "but I know that Bob doesn't know either."
> 3. Bob: "At first I didn’t know when Cheryl's birthday is...
> 4. Bob: "but now I know."
> 5. Alice: "Now I know when Cheryl's birthday is."
>
> When is Cheryl's birthday?

_(This is actually our in-house variant of the [original puzzle](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday). It is logically the same, but we find this variant more fun and easier to explain to people, because it doesn't rely on positing an arbitrary subset of dates.)_

We will progressively build up a model of this scenario this in memo by writing a model for each of the 5 utterances. To model whether someone knows for certain when Cheryl's birthday is, we will check whether according to them the variance over the date is zero.

In [1]:
from memo import memo, domain
import jax
import jax.numpy as np
from enum import IntEnum

class Month(IntEnum):
    February = 0
    March = 1
    April = 2

Day = np.arange(1, 31 + 1)

class U(IntEnum):
    DUNNO = 0
    KNOWN = 1

@jax.jit
def possible(m, d):  # 31 days hath...
    return d <= np.array([29, 31, 30])[m]

Now, let's build up this model step by step.

Alice: (after observing month $m$) "I don't know when Cheryl's birthday is."

In [2]:
@memo
def a_u1[m: Month, u: U]():
    a: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d))
    ]
    a: observes [c.m] is m
    return u == a[Var[c.d] == 0]  # note: Alice's variance over Cheryl's d
# print(a_u1())

Alice: "...but I know that Bob doesn't know either."

In [3]:
@memo
def a_u2[m: Month, u: U]():
    a: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d)),
        b: thinks[
            c: chooses(m in Month, wpp=1),
            c: chooses(d in Day, wpp=possible(m, d))
        ],
        b: observes [c.d] is c.d
    ]
    a: observes [c.m] is m
    return u == a[Pr[b[Var[c.m] == 0]] > 0]
# print(a_u2())

Bob: "At first I didn't know..." (similar to `a_u1`)

In [4]:
@memo
def b_u3[d: Day, u: U]():
    b: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d))
    ]
    b: observes [c.d] is d
    return u == b[Var[c.m] == 0]
# print(b_u3())

Bob: "But now..." (conditions on result of `a_u1` and `a_u2`!)

In [5]:
@memo
def b_u4[d: Day, u1: U, u2: U, u: U]():
    b: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d)),
        a: thinks[
            c: chooses(m in Month, wpp=1),
            c: chooses(d in Day, wpp=possible(m, d)),
        ],
        a: observes [c.m] is c.m,
        a: chooses(u1 in U, wpp=a_u1[c.m, u1]()),
        a: chooses(u2 in U, wpp=a_u2[c.m, u2]())
    ]
    b: observes [c.d] is d
    b: observes [a.u1] is u1
    b: observes [a.u2] is u2
    return u == b[Var[c.m] == 0]
# print(b_u4()[:, U.DUNNO, U.DUNNO])

Alice: "Now I know." (conditions on `b_u3` and `b_u4`)

In [6]:
@memo
def a_u5[m: Month, u1: U, u2: U, u3: U, u4: U, u: U]():
    a: knows(u1, u2)
    a: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d)),
        b: thinks[
            c: chooses(m in Month, wpp=1),
            c: chooses(d in Day, wpp=possible(m, d))
        ],
        b: knows(u1, u2),
        b: observes [c.d] is c.d,
        b: chooses(u3 in U, wpp=b_u3[c.d, u3]()),
        b: chooses(u4 in U, wpp=b_u4[c.d, u1, u2, u4]()),
    ]
    a: observes [c.m] is m
    a: observes [b.u3] is u3
    a: observes [b.u4] is u4
    return u == a[Var[c.d] == 0]
# a_u5()[:, U.DUNNO, U.DUNNO, U.DUNNO, U.KNOWN]

Putting everything together, we condition on all 5 utterances.

In [7]:
@memo
def puzzle[m: Month, d: Day, u1: U, u2: U, u3: U, u4: U, u5: U]():
    z: thinks[
        c: chooses(m in Month, wpp=1),
        c: chooses(d in Day, wpp=possible(m, d)),
        c: chooses(u1 in U, wpp=a_u1[m, u1]()),
        c: chooses(u2 in U, wpp=a_u2[m, u2]()),
        c: chooses(u3 in U, wpp=b_u3[d, u3]()),
        c: chooses(u4 in U, wpp=b_u4[d, u1, u2, u4]()),
        c: chooses(u5 in U, wpp=a_u5[m, u1, u2, u3, u4, u5]()),
    ]
    z: observes [c.u1] is u1
    z: observes [c.u2] is u2
    z: observes [c.u3] is u3
    z: observes [c.u4] is u4
    z: observes [c.u5] is u5
    z: knows(m, d)
    return z[E[c.m == m and c.d == d]]

answer = puzzle()[:, :, U.DUNNO, U.DUNNO, U.DUNNO, U.KNOWN, U.KNOWN]

Finally, we extract the answer by finding the nonzero entry in the inferred $(m, d)$.

In [8]:
for m in Month:
    for di, d in enumerate(Day):
        if answer[m, di]:
            print(m.name, d)

April 30
